In [5]:
################################## Importações para trabalhar com watson IBM ##################################
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions, KeywordsOptions
from ibm_watson import ApiException
# from botocore.client import Config
# import ibm_boto3

################################## Importações ##################################
# import types 
import pandas as pd
import re
import string

################################# Importações Google YouTube API #################################
from googleapiclient.discovery import build
import googleapiclient.discovery
#base de dados:
#https://www.kaggle.com/tanulsingh077/twitter-sentiment-extaction-analysis-eda-and-model/output?select=submission.csv

#teste:
#https://natural-language-understanding-demo.ng.bluemix.net/

In [172]:
################################## Configurações serviço IBM ##################################
#realiza autenticação com servidor IBM
authenticator = IAMAuthenticator('LsRvjTiGO2hm5CmMOkYumONXX1x08ej0nZED2cRZYvqM')
service = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

# #seleciona o serviço de processamento de linguagem natural
service.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/fe0885df-7c76-4192-9bc8-6c6d7ca836ae')

In [173]:

################################## INÍCIO FUNÇÕES ##################################

def sentiment_and_emotion(st,service=service):
    """
    Função para fazer requisição com o servidor IBM e obter emoção e sentimento do texto a ser analisado
    return: JSON com análises do texto informado
    """
    try:
        return (service.analyze(text=st,features=Features(emotion=EmotionOptions(),sentiment=SentimentOptions())).get_result())
    except ApiException as ex:
        return "NULL"
    #analisa a emoção do texto contido na variável st
    #return (service.analyze(text=st,features=Features(emotion=EmotionOptions())).get_result())
    

def sentiment_extractor(row_num):
    """
    Função para pegar o score do sentimento e colocar em uma nova coluna dataframe
    return: score do sentimento
    """    
    try:
        sentiment=filtrado['sentiment_emotion_json'][row_num]['sentiment']['document']['label']
        return sentiment
    except:
        return "NULL"
################################## FIM FUNÇÕES ##################################

In [174]:
def clean_text(text):
    """
    Função para remover caracteres especiais, http...
    """
    text = str(text).lower()
    text = str(text).strip()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)  
    
    return text

def emotion_extractor(row_num):
    """
    Função para pegar o score do sentimento e colocar em uma nova coluna dataframe
    return: score do sentimento
    """    
    #sentiment_scores=df2['sentiment_keyword_json'][row_num]['sentiment']['document']
    try:
        dictResult = filtrado['sentiment_emotion_json'][row_num]['emotion']['document']['emotion']
        dictResult = [max(dictResult, key=dictResult.get)]
        return dictResult[0]
    except:
        return "NULL"

In [175]:
#leitura da base de dados
df_data_1 = pd.read_csv('database/databaseCommentsTwitter.csv')
#limpeza de caracteres especiais, pontuação, links, trim...
df_data_1['selected_text'] = df_data_1['selected_text'].apply(lambda row : clean_text(row))

totalComentarios = df_data_1['selected_text'].count()
print("Total de comentários: ", totalComentarios)

Total de comentários:  3534


In [176]:
################## FIX: CLASSIFICAR TODOS OS DADOS... ##########

#requisição com servidor
#parâmetros:
    #texto
    #eixo para adicionar a coluna
df1 = df_data_1.head(30)
df1['sentiment_emotion_json'] = df1.apply(lambda row : sentiment_and_emotion(str(row['selected_text'])),axis = 1)

#df_teste = pd.read_csv('database/testeDadosClassificados.csv')

#print(df_teste)

<class 'pandas.core.frame.DataFrame'>


<ipython-input-176-9daa14734a7f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentiment_emotion_json'] = df1.apply(lambda row : sentiment_and_emotion(str(row['selected_text'])),axis = 1)


In [177]:
df2 = df1[['selected_text','sentiment_emotion_json']]
df2.dropna(inplace=True)

<ipython-input-177-d75cf66d643f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


In [178]:
#obter somente o label
#df2['sentiment_emotion_json'][1]['sentiment']['document']['label']
#df2['sentiment_keyword_json'][0]['sentiment']

filtrado = df2[df2['sentiment_emotion_json'] != 'NULL']
filtrado.dropna(inplace=True)
filtrado = filtrado.reset_index()
filtrado = filtrado.drop(columns=['index'])
#print(filtrado['sentiment_emotion_json'][0]['emotion']['document']['emotion'])
filtrado['sentiment'] = filtrado.apply(lambda row : sentiment_extractor(row.name),axis = 1)
filtrado['emotion'] = filtrado.apply(lambda row : emotion_extractor(row.name),axis = 1)
#filtrado2 = filtrado[filtrado['sentiment_emotion_json'] != 'NULL']
print(filtrado)

#dictTeste = filtrado['sentiment_emotion_json'][0]['emotion']['document']['emotion']
#dictTeste = [max(dictTeste, key=dictTeste.get)]
#print(dictTeste[0])


                                        selected_text  \
0                           last session of the day     
1                                            exciting   
2                                          happy bday   
3   and within a short time of the last clue all o...   
4   what did you get  my day is alright havent don...   
5   my bike was put on holdshould have known that ...   
6                             i checked  we didnt win   
7    and youre on twitter did the tavern bore you ...   
8   i feel like my phones hole is not a virgin tha...   
9   im going into a spiritual stagnentation its ex...   
10  my dead grandpa pays more attention to me than...   
11                               about to go to sleep   
12                                      thats so cool   
13  hey peoples dont you just hate being grounded ...   
14               huh another scarepoint coding sunday   
15                  look who i found just for you       
16  no ac the fan doesnt swing 

<ipython-input-178-bf5bc3d6be94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtrado.dropna(inplace=True)


In [182]:
#####salvando dados classificados em csv
saveFile = filtrado.drop(columns=['sentiment_emotion_json'])
saveFile.to_csv("database/testeDadosClassificados.csv", encoding='utf-8', index=False)

In [2]:
readFile = pd.read_csv('database/testeDadosClassificados.csv')

In [3]:
readFile

,selected_text,sentiment,emotion
0,last session of the day,neutral,sadness
1,exciting,positive,joy
2,happy bday,positive,joy
3,and within a short time of the last clue all o...,negative,sadness
4,what did you get my day is alright havent don...,negative,sadness
5,my bike was put on holdshould have known that ...,negative,fear
6,i checked we didnt win,negative,joy
7,and youre on twitter did the tavern bore you ...,negative,sadness
8,i feel like my phones hole is not a virgin tha...,negative,sadness
9,im going into a spiritual stagnentation its ex...,positive,joy


In [34]:
############################ LER COMENTÁRIOS YOUTUBE ############################
readApiKey = pd.read_csv('apiYouTubeKey.txt', sep="\t")

youTubeApiKey = str(readApiKey[readApiKey['api'] == 'YouTube']['key'][0])
#youtube = build('youtube','v3', developerKey=youTubeApiKey)
video_id = "UU6v80aDMV4"

In [69]:
def video_comments(video_id): 
    # empty list for storing reply 
    replies = [] 
  
    # creating youtube resource object 
    youtube = build('youtube', 'v3', 
                    developerKey=api_key) 
  
    # retrieve youtube video results 
    video_response=youtube.commentThreads().list( 
    part='snippet,replies', 
    videoId=video_id 
    ).execute() 
  
    # iterate video response 
    while video_response: 
        
        # extracting required info 
        # from each result object  
        for item in video_response['items']: 
            
            # Extracting comments 
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay'] 
              
            # counting number of reply of comment 
            replycount = item['snippet']['totalReplyCount'] 
  
            # if reply is there 
            if replycount>0: 
                
                # iterate through all reply 
                for reply in item['replies']['comments']: 
                    
                    # Extract reply 
                    reply = reply['snippet']['textDisplay'] 
                      
                    # Store reply is list 
                    replies.append(reply) 
  
            # print comment with list of reply 
            print(comment, replies, end = '\n\n') 
  
            # empty reply list 
            replies = [] 
  
        # Again repeat 
        if 'nextPageToken' in video_response: 
            video_response = youtube.commentThreads().list( 
                    part = 'snippet,replies', 
                    videoId = video_id 
                ).execute() 
        else: 
            break
  
# Enter video id 
video_id = "pxofwuWTs7c"
  
# Call function 
video_comments(video_id)

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

...

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

super []

any tutorials using Python 3+? []

This is so helpful! []

I won’t stop testifying for <b>smithjuli</b> on telegram, I’m always happy to deal with her 💯✅ []

need itpl support []

I tried to run this but you cant register for APIs because godaddy is not allowing any more sign ups. the key @Ethaisa123 provided is no longer active so codingentrepreneurs would have deactivated it. Damn! []

here is the key that he uses: 9fb8cd34cab8e83690473133f51943b5c93f []

please let me know the terminal font name, I&#39;m in pain searching it all over.... Thanks man. []

Ok cool! []

excellent tutorial! thanks! []

I just discovered a new world in coding, APIs are a miracle []

Helpful []

nice example and easily explained. thanks! []

I don&#39;t see a link for signup on the locu site?  Just login... []

I&#39;m not getting any link for signup. It&#39;s just showing only for login. Please revisit the website and please mail me on (shubhambansal5642@gmail.com), how to signup. []

Can you us

KeyboardInterrupt: 

In [75]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


service = build('youtube', 'v3', developerKey=api_key) 

response = service.commentThreads().list(
                part = 'snippet',
                videoId = "UU6v80aDMV4",
                maxResults = 100, # Only take top 100 comments...
                order = 'relevance', #... ranked on relevance
                textFormat = 'plainText',
                ).execute()

comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []
for item in response['items']:
    print(item['snippet']['topLevelComment']['snippet']['textDisplay'])
    comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
    comment_id_temp.append(item['snippet']['topLevelComment']['id'])
    reply_count_temp.append(item['snippet']['totalReplyCount'])
    like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
comments_pop.extend(comments_temp)
comment_id_pop.extend(comment_id_temp)
reply_count_pop.extend(reply_count_temp)
like_count_pop.extend(like_count_temp)

video_id_pop.extend([video_id[i]]*len(comments_temp))
#channel_pop.extend([channel[i]]*len(comments_temp))
#video_title_pop.extend([video_title[i]]*len(comments_temp))
#video_desc_pop.extend([video_desc[i]]*len(comments_temp))

Muito bom!!! Já deixando meu comentário positivo kkkk :)
Curti essa Library, rolava sequencia em, parece ser muito interessante
Poderia trazer mais conteúdo sobre o PyGames. E alguns outros frameworks do Python. 

Abraço .
Em todo momento me veio a voz do bender nos "ahaaaa, agora eu entendi, agora dos as peças se encaixaram"
Novo quadro do canal, "Lendo comentários aleatoriamente"
Parabéns , principalmente por passar uma "metodologia" para chegar ao objetivo .:) Vai liberar o código ?
Queria ser inteligente como vc em programas, muito bom o vídeo
Parabéns mano. Conteúdo de ótima qualidade. Muito obrigado!
Show de video.. Valeu. ✌️✌️✌️
Otimo video!
Muito bacana a ideia de mostrar os bastidores de forma acelerada, pois mostra que programação, como qualquer outra profissão exige convicção,  persistência, dente outros, para termos sucesso! Excelente vídeo!
Faz um jogo com Pygame por favor !!!!!!
Muito foda,  merecia muito mais views
Por favor faz um vídeo de como fazer duas linguagem de p